### Imports

In [ ]:
import requests
from bs4 import BeautifulSoup

1. Send a request for accesing first webpage with opinions about a product.


In [ ]:
url = "https://www.ceneo.pl/167976636;02514#tab=reviews"
response = requests.get(url)

2. If response is ok, parse HTML page content into DOM structure.


In [ ]:
if response.status_code == 200:
    page_dom = BeautifulSoup(response.text, "html.parser")

3. Extract opinions and their components from DOM structure 


In [ ]:
opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
print(type(page_dom))
print(type(opinions))
print(len(opinions))
opinions=opinions.pop(0)
print(type(opinions))